In [7]:
# 导入数据集
from utils.dataset import Argo3DTemperatureDataset, FrameType

data_set = Argo3DTemperatureDataset()
print(len(data_set))
print(data_set.shape())

train_data = data_set.getFrame(FrameType.surface, time=(0, 20), lon=(160, 180), lat=(60, 80), depth=(0, 1))


Reading data file: D:/ArgoDataset\BOA_Argo_2004_01.nc
Reading data file: D:/ArgoDataset\BOA_Argo_2004_02.nc
Reading data file: D:/ArgoDataset\BOA_Argo_2004_03.nc
Reading data file: D:/ArgoDataset\BOA_Argo_2004_04.nc
Reading data file: D:/ArgoDataset\BOA_Argo_2004_05.nc
Reading data file: D:/ArgoDataset\BOA_Argo_2004_06.nc
Reading data file: D:/ArgoDataset\BOA_Argo_2004_07.nc
Reading data file: D:/ArgoDataset\BOA_Argo_2004_08.nc
Reading data file: D:/ArgoDataset\BOA_Argo_2004_09.nc
Reading data file: D:/ArgoDataset\BOA_Argo_2004_10.nc
Reading data file: D:/ArgoDataset\BOA_Argo_2004_11.nc
Reading data file: D:/ArgoDataset\BOA_Argo_2004_12.nc
Reading data file: D:/ArgoDataset\BOA_Argo_2005_01.nc
Reading data file: D:/ArgoDataset\BOA_Argo_2005_02.nc
Reading data file: D:/ArgoDataset\BOA_Argo_2005_03.nc
Reading data file: D:/ArgoDataset\BOA_Argo_2005_04.nc
Reading data file: D:/ArgoDataset\BOA_Argo_2005_05.nc
Reading data file: D:/ArgoDataset\BOA_Argo_2005_06.nc
Reading data file: D:/ArgoDa